# 1. nfcore/rnaseq workflow

From the [nfcore/rnaseq website](https://nf-co.re/rnaseq/3.12.0):

> nf-core/rnaseq is a bioinformatics pipeline that can be used to analyse RNA sequencing data obtained from organisms with a reference genome and annotation. It takes a samplesheet and FASTQ files as input, performs quality control (QC), trimming and (pseudo-)alignment, and produces a gene expression matrix and extensive QC report.

> The pipeline is built using [Nextflow](https://www.nextflow.io/), a workflow tool to run tasks across multiple compute infrastructures in a very portable manner.

*************

# Contents

[1a. Prerequisites](#prereq)

[1b. nfcore/rnaseq workflow overview](#workflow)

[1c. Running a test dataset](#test)

[1d. Initial setup](#setup)

[1e. Creating a samplesheet](#sampsheet)

[1f. Quality assessment](#qc)

******************

## 1a. Prerequisites <a class="anchor" id="prereq2"></a>

To run the nfcore/rnaseq workflow, you will need:

1. **A QUT HPC account.** If you are seeing this Notebook, you most likely already have a HPC account. Regardless, you can request an account be created, or request any other HPC or bioinformatics support, via the portal here: https://eresearchqut.atlassian.net/servicedesk/customer/portals

2. Nextflow needs to be installed on your HPC account. **If you haven't already installed Nextflow, or you need to update Nextflow to the latest version, click on the following link, which will download and open a Jupyter Notebook that will step you through installing Nextflow:** https://jupyterhub.eres.qut.edu.au/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Feresearchqut%2FJupyter_Nextflow_install&urlpath=lab%2Ftree%2FJupyter_Nextflow_install%2FNextflow.ipynb&branch=main

3. **Your sequence data files** (fastq.gz format) copied to a location on the QUT HPC that you can access. If you are unsure of the location of your data files, submit a service request through the portal (link above).

**********

## 1b. nfcore/rnaseq workflow overview <a class="anchor" id="workflow"></a>

nfcore/rnaseq is a [Nextflow](https://www.nextflow.io/) workflow for analysing RNA-Seq data. Nextflow is a workflow management tool. [nfcore](https://nf-co.re/) is a repository of curated [bioinformatics pipelines](https://nf-co.re/pipelines).

For workflow information: https://nf-co.re/rnaseq/3.12.0

How to run workflow: https://nf-co.re/rnaseq/3.12.0/docs/usage

Workflow output: https://nf-co.re/rnaseq/3.12.0/docs/output

![](https://raw.githubusercontent.com/nf-core/rnaseq/3.12.0//docs/images/nf-core-rnaseq_metro_map_grey.png)

The pipeline is built using Nextflow and processes data using the following steps:

* [Preprocessing](https://nf-co.re/rnaseq/3.10.1/docs/output#preprocessing)

  * [cat](https://nf-co.re/rnaseq/3.10.1/docs/output#cat) - Merge re-sequenced FastQ files

  * [FastQC](https://nf-co.re/rnaseq/3.10.1/docs/output#fastqc) - Raw read QC

  * [UMI-tools extract](https://nf-co.re/rnaseq/3.10.1/docs/output#umi-tools-extract) - UMI barcode extraction

  * [TrimGalore](https://nf-co.re/rnaseq/3.10.1/docs/output#trimgalore) - Adapter and quality trimming

  * [BBSplit](https://nf-co.re/rnaseq/3.10.1/docs/output#bbsplit) - Removal of genome contaminants

  * [SortMeRNA](https://nf-co.re/rnaseq/3.10.1/docs/output#sortmerna) - Removal of ribosomal RNA

* [Alignment and quantification](https://nf-co.re/rnaseq/3.10.1/docs/output#alignment-and-quantification)

  * [STAR and Salmon](https://nf-co.re/rnaseq/3.10.1/docs/output#star-and-salmon) - Fast spliced aware genome alignment and transcriptome quantification

  * [STAR via RSEM](https://nf-co.re/rnaseq/3.10.1/docs/output#star-via-rsem) - Alignment and quantification of expression levels

  * [HISAT2](https://nf-co.re/rnaseq/3.10.1/docs/output#hisat2) - Memory efficient splice aware alignment to a reference

* [Alignment post-processing](https://nf-co.re/rnaseq/3.10.1/docs/output#alignment-post-processing)

  * [SAMtools](https://nf-co.re/rnaseq/3.10.1/docs/output#samtools) - Sort and index alignments

  * [UMI-tools dedup](https://nf-co.re/rnaseq/3.10.1/docs/output#umi-tools-dedup) - UMI-based deduplication

  * [picard MarkDuplicates](https://nf-co.re/rnaseq/3.10.1/docs/output#picard-markduplicates) - Duplicate read marking

* [Other steps](https://nf-co.re/rnaseq/3.10.1/docs/output#other-steps)

  * [StringTie](https://nf-co.re/rnaseq/3.10.1/docs/output#stringtie) - Transcript assembly and quantification

  * [BEDTools and bedGraphToBigWig](https://nf-co.re/rnaseq/3.10.1/docs/output#bedtools-and-bedgraphtobigwig) - Create bigWig coverage files

* [Quality control](https://nf-co.re/rnaseq/3.10.1/docs/output#quality-control)

  * [RSeQC](https://nf-co.re/rnaseq/3.10.1/docs/output#rseqc) - Various RNA-seq QC metrics

  * [Qualimap](https://nf-co.re/rnaseq/3.10.1/docs/output#qualimap) - Various RNA-seq QC metrics

  * [dupRadar](https://nf-co.re/rnaseq/3.10.1/docs/output#dupradar) - Assessment of technical / biological read duplication

  * [Preseq](https://nf-co.re/rnaseq/3.10.1/docs/output#preseq) - Estimation of library complexity

  * [featureCounts](https://nf-co.re/rnaseq/3.10.1/docs/output#featurecounts) - Read counting relative to gene biotype

  * [DESeq2](https://nf-co.re/rnaseq/3.10.1/docs/output#deseq2) - PCA plot and sample pairwise distance heatmap and dendrogram

  * [MultiQC](https://nf-co.re/rnaseq/3.10.1/docs/output#multiqc) - Present QC for raw reads, alignment, read counting and sample similiarity

* [Pseudo-alignment and quantification](https://nf-co.re/rnaseq/3.10.1/docs/output#pseudo-alignment-and-quantification)

  * [Salmon](https://nf-co.re/rnaseq/3.10.1/docs/output#salmon) - Wicked fast gene and isoform quantification relative to the transcriptome

* [Workflow reporting and genomes](https://nf-co.re/rnaseq/3.10.1/docs/output#workflow-reporting-and-genomes)

  * [Reference genome files](https://nf-co.re/rnaseq/3.10.1/docs/output#reference-genome-files) - Saving reference genome indices/files

  * [Pipeline information](https://nf-co.re/rnaseq/3.10.1/docs/output#pipeline-information) - Report metrics generated during the workflow execution

****************

## 1c. Running a test dataset <a class="anchor" id="test"></a>

This section will run a small test dataset through the nfcore/sarek workflow, to see if you can successfully run nfcore/sarek.

<div class="alert alert-block alert-warning">
Run the following code cell to run the nfcore/sarek test. Note that this is set to run without output messages (`-q`) as these will span multiple pages. Instead `nextflow log -f status` is run after the test run has finished, to see if the job ran sucessfully.
</div>

<div class="alert alert-block alert-danger">
This test can take an hour or more to complete. You don't have to run this test before you run the full workflow, but you may want to run it to test that the workflow will run on your system. 
</div>

In [ ]:
mkdir $HOME/nftemp && cd $HOME/nftemp
module load java
nextflow -q run nf-core/rnaseq -r 3.12.2 --outdir test -profile test,singularity
nextflow log -f status

**In the last 3 lines you should see a table with `TIMESTAMP` .. `DURATION`, etc (You can ignore any `WARN` messages). If the `STATUS` is `OK` then the test run was successful and you can go to the next section. If you don't see `STATUS` being `OK`, submit a support request through the [Portal](https://eresearchqut.atlassian.net/servicedesk/customer/portals).** 

<div class="alert alert-block alert-success">
To run the above test, Nextflow downloads test data and generates numerous output directories and files. These should be removed after the test is run, by running the following:  
    </div>

In [ ]:
cd $HOME
rm -rf nftemp

*****************************

## 1d. Initial setup <a class="anchor" id="setup"></a>

<div class="alert alert-block alert-info">
Enter the directory that contains your sequence data files. You can find this directory path by typing 'pwd' on the command-line when you are in that directory, or by contacting the HPC staff via the [portal](https://eresearchqut.atlassian.net/servicedesk/customer/portals). The structure of the below command should be `root_path=/directory/containing/my/vcf/files`.
</div>

In [ ]:
root_path=/work/eresearch_bio/nextflow/rnaseq_test/data

<div class="alert alert-block alert-success">
Now move to the above directory (cd = change directory): 
</div>

In [ ]:
cd $root_path

<div class="alert alert-block alert-danger">
NOTE: the above two code cells must be run every time you use this Notebook.
    </div>

<div class="alert alert-block alert-warning">
To see if you are in the correct directory, run the 'ls' code cell below. You should see a list of all your sample files ('...fastq.gz' or '...fq.gz' files). If you don't see the files, you've entered the above location incorrectly and need to correct and re-run the above code cell.
</div>

In [ ]:
ls

********

## 1e. Creating a samplesheet <a class="anchor" id="sampsheet"></a>

nfcore/sarek requires a samplesheet, that contains, at minumum, 4 columns:
1. sample IDs
2. Read 1 sequence data files (fastq files)
3. Read 2 sequence data files (fastq files)
4. [Strandedness](https://www.rna-seqblog.com/what-is-strandedness-in-rna-seq-data/)
Details about the required samplesheet structure, as well as an example samplesheet, can be see [here](https://nf-co.re/rnaseq/3.12.0/docs/usage#samplesheet-input).

<div class="alert alert-block alert-danger">
FASTQ FILE NAMING CONVENTIONS: sequence data files (fastq files) are typically paired-end, with two fastq files per sample, Read 1 (R1) and Read 2 (R2). In addition, fastq files are typically gzipped ('filename.fastq.gz' - nfcore/rnaseq requirs this) and may be named 'fastq' or just 'fq'. This results in a few different naming patterns. To create your samplesheet correctly, you need to enter the naming patterns that match your fastq files in the code cells below (i.e. the last few characters at the end of each file that are teh same in ALL Read 1 and ALL Read 2 files respectively). Some examples of fastq naming conventions can be `R1_001.fastq.gz`, `1_fq.gz`, `R2_fq.gz`, `2_fastq.gz', etc.  
</div>

For more information about fastq files, see [here](https://knowledge.illumina.com/software/general/software-general-reference_material-list/000002211).

<div class="alert alert-block alert-info">
First, input the common naming pattern of your Read 1 data files. You can see this from the `ls` command above.
</div>

In [ ]:
fq1_format=_merged_R1.fastq.gz

<div class="alert alert-block alert-info">
Enter the Read 2 files naming pattern.
</div>

In [ ]:
fq2_format=_merged_R2.fastq.gz

<div class="alert alert-block alert-success">
Now you generate the samplesheet by running the following:
</div>

In [ ]:
cd $root_path
module load python/3.10.8-gcccore-12.2.0
wget -L -O fastq_dir_to_samplesheet.py https://raw.githubusercontent.com/nf-core/rnaseq/master/bin/fastq_dir_to_samplesheet.py
chmod +x fastq_dir_to_samplesheet.py
./fastq_dir_to_samplesheet.py $root_path samplesheet.csv \
    --strandedness auto \
    --read1_extension $fq1_format \
    --read2_extension $fq2_format

<div class="alert alert-block alert-success">
View the samplesheet by running the below cell. The structure should look the same as the example [here](https://nf-co.re/rnaseq/3.12.0/docs/usage#samplesheet-input):
</div>

In [ ]:
cat samplesheet.csv

******************

# 1f. Quality assessment <a class="anchor" id="qc"></a>

In this section you will generate fastq quality reports, which will inform you of the quality of your data and any specific quality filtering parameters you should apply (next section).

<div class="alert alert-block alert-danger">
The code in this section submits your workflow to the HPC as a 'job' in the queue. If you have many samples, this may take many hours or even days to fully process. We provide a code cell at the end of this section to monitor your job.
</div>

See here for information on HPC job submission: https://eresearchqut.atlassian.net/wiki/spaces/EG/pages/1545143157/HPC

<div class="alert alert-block alert-info">
Choose a genome for your target organism. For model organisms this is the iGenomes ID. E.g. human could be `genome=GRCh38`, mouse `genome=GRCm38`, etc. See here for details: https://github.com/ewels/AWS-iGenomes
</div>

In [ ]:
genome=GRCh38

<div class="alert alert-block alert-success">
Run the following code cell to create the launch script that will submit your job. Note the '--skip_...' lines. This will run the nfcore/rnaseq workflow, while skipping the main analysis steps (i.e. running just the quality assessment).
</div>

In [ ]:
cat <<EOF > qc_launch.pbs
#!/bin/bash -l
#PBS -N nfrnaqc
#PBS -l select=1:ncpus=2:mem=4gb
#PBS -l walltime=24:00:00
cd $root_path
module load java
NXF_OPTS='-Xms1g -Xmx4g'
nextflow run nf-core/rnaseq \
-profile singularity \
      -r 3.12.0 \
      --input samplesheet.csv \
      --outdir results \
      --genome $genome \
      --skip_trimming \
      --skip_alignment \
      --skip_pseudo_alignment
EOF

<div class="alert alert-block alert-success">
Now submit this as a HPC job.
</div>

In [ ]:
qsub qc_launch.pbs

<div class="alert alert-block alert-info">
You can monitor the progress of your job by running 'qstat'. This will give you information about your job. Enter your job ID (seen above, "xxxxxxx.pbs") in the qstat command below:
</div>

In [ ]:
qstat 4928274.pbs

In the 'S' column, 'Q' means your job is still in the queue, 'R' means it's running, and if your job is complete it will say "Job has finished". Once the job is complete, you can view the quality reports.

<div class="alert alert-block alert-success">

</div>